In [1]:
import torch
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split

from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, AutoModelForSequenceClassification, Trainer, \
EarlyStoppingCallback, pipeline, set_seed, PretrainedConfig, EvalPrediction

In [4]:
data = DatasetDict.load_from_disk('data/train_valid_split_prepared')
trend_names = pd.read_csv('data/trend_names.csv')['0'].to_list()
id2label = {k:v for k, v in enumerate(trend_names)}
label2id = {v:k for k, v in id2label.items()}
labels = list(label2id.keys())

In [5]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'assessment', 'tags', 'text', 'Долгая доставка', 'Доставка стала долгой', 'Время доставки не соответствует заявленому', 'Регулярные опоздания', 'Не отследить реальное время доставки', 'Курьер на карте', 'Нет доставки по адресу', 'Не предупреждаем об удалении товара', 'Высокая минимальная сумма заказа', 'Сумма заказа меняется во время набора корзины', 'Минимальная сумма заказа', 'Товары с подходящим сроком годности', 'Высокие цены', 'Не довезли товар', 'Товар испорчен во время доставки', 'Просроченные товары', 'Замечания по работе курьеров', 'Не читаем комментарии', 'Спасибо', 'Нет смысла', 'Всё нормально', 'Всё плохо', 'Скидки для постоянных клиентов', 'Больше акций/скидок', 'Скидка/промокод распространяется не на все товары', 'Непонятно как работает скидка', 'Не сработала скидка/акция/промокод', 'Качество товаров', 'Маленький ассортимент', 'Нет в наличии товара', 'Качество поддержки', 'Замечания по работе сбо

**Примечания**:
- За кадром остались эксперименты с разбивкой на обучающую/валидационную выборку, но лучшее качество на тесте показало обучение на всем датасете. Просто модель видела больше разнообразных данных.

In [6]:
con_data = concatenate_datasets([data['train'], data['validation']])
data['train'] = con_data

In [7]:
con_data

Dataset({
    features: ['Unnamed: 0', 'index', 'assessment', 'tags', 'text', 'Долгая доставка', 'Доставка стала долгой', 'Время доставки не соответствует заявленому', 'Регулярные опоздания', 'Не отследить реальное время доставки', 'Курьер на карте', 'Нет доставки по адресу', 'Не предупреждаем об удалении товара', 'Высокая минимальная сумма заказа', 'Сумма заказа меняется во время набора корзины', 'Минимальная сумма заказа', 'Товары с подходящим сроком годности', 'Высокие цены', 'Не довезли товар', 'Товар испорчен во время доставки', 'Просроченные товары', 'Замечания по работе курьеров', 'Не читаем комментарии', 'Спасибо', 'Нет смысла', 'Всё нормально', 'Всё плохо', 'Скидки для постоянных клиентов', 'Больше акций/скидок', 'Скидка/промокод распространяется не на все товары', 'Непонятно как работает скидка', 'Не сработала скидка/акция/промокод', 'Качество товаров', 'Маленький ассортимент', 'Нет в наличии товара', 'Качество поддержки', 'Замечания по работе сборщика', 'Отменили заказ', 'Зн

В качестве базовой модели возьмем модель от ВК - 'deepvk/USER-bge-m3', которая в свою очередь является облегченной и дообученной версией одного из лучших энкодеров - https://huggingface.co/BAAI/bge-m3

In [8]:
model_link = 'deepvk/USER-bge-m3'

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_link)

def preprocess_data(examples, text_col='text', max_len=128):
  # take a batch of texts
  text = examples[text_col]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=max_len)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [10]:
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    # print(predictions.shape)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs > threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    accuracy_0 = accuracy_score(y_true[:, 0], y_pred[:, 0])
    accuracy_1 = accuracy_score(y_true[:, 1], y_pred[:, 1])
    accuracy_2 = accuracy_score(y_true[:, 2], y_pred[:, 2])
    accuracy_3 = accuracy_score(y_true[:, 3], y_pred[:, 3])
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy,
              'accuracy_0': accuracy_0,
              'accuracy_1': accuracy_1,
              'accuracy_2': accuracy_2,
              'accuracy_3': accuracy_3}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

**Примечания**:
- за кадром остались эксперименты с различным форматом текстовых данных. Имея информацию от орагнизаторов, что assessment - это оценка пользователя, добавим ее к отзыву и будем учиться на этом. Этот сценарий показывал наиболее высокую accuracy на тесте.
- поэтому выбираем фичу text_mark.

In [11]:
TEXT_COL = 'text_mark'

In [12]:
encoded_dataset = data.map(preprocess_data, batched=True, remove_columns=data['train'].column_names, fn_kwargs={'text_col': TEXT_COL})
encoded_dataset.set_format("torch")

Map:   0%|          | 0/4623 [00:00<?, ? examples/s]

Map:   0%|          | 0/463 [00:00<?, ? examples/s]

In [13]:
encoded_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4623
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 463
    })
})

In [14]:
model = AutoModelForSequenceClassification.from_pretrained(model_link, 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id,
                                                          )

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at deepvk/USER-bge-m3 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**Примечания**:
- модель училась на 4-х картах 2080.

In [16]:
MODEL = 'deep_vk_bge' 
DATA = 'ecom'
EPOCHS = 18 
lr = 5e-5 
BATCH_SIZE = 8 
weight_decay = 1e-6
TEST_SIZE = 0.1

name_fine_tune = f'{MODEL}_{DATA}_{EPOCHS}_{BATCH_SIZE}_{TEST_SIZE}_{lr}_{weight_decay}_{TEXT_COL}'

training_args = TrainingArguments( 
                                  output_dir=name_fine_tune,
                                  num_train_epochs=EPOCHS, 
                                  weight_decay=weight_decay, 
                                  per_device_train_batch_size=BATCH_SIZE,
                                  learning_rate=lr, 
                                  logging_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="accuracy",
                                  save_strategy='steps',
                                  eval_strategy="steps",
                                  report_to='none',
                                  push_to_hub=False,
                                  run_name=name_fine_tune,
                                  seed=42,
                                  data_seed=42,
                                  fp16=False
                                  )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [17]:
trainer = Trainer( 
    model,
    training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [18]:
name_fine_tune

'deep_vk_bge_ecom_18_8_0.1_5e-05_1e-06_text_mark'

Качество валидационной выборки здесь просто для информации. За указанное количество эпох модель почти выучивает всю обучающую выборку.

In [19]:
trainer.train()

/home/valukhov_ao/projects/jupyter_venv/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

Обученную модель отправим на hf, чтобы к ней был доступ

In [22]:
trainer.model.push_to_hub('bge-ecom-trends-classifier')

https://huggingface.co/Maldopast/bge-ecom-trends-classifier

**Примечания**:

Пробовал, но не дало эффекта

- За кадром остались также эксперименты другими архитектурами моделей (XLM-Roberta - лучшая);
- Другими лосс-функциями - reduction='sum' в BCEWithLogitsLoss, добавление весов к классам, использование focal loss